In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyamg.classical import ruge_stuben_solver

In [ ]:
from topo import SHex
from mesh import Mesh, uniform_nodes_3d
from basis import LagrangeBasisHex, LobattoBasisHex
from poisson import poisson_Kloc
from assemble import simple_build_rhs
from cyassemble import simple_assembly

In [ ]:
order = 4
L = (1.0,1.0,1.0)
x_max = L[0]
y_max = L[1]
z_max = L[2]
periodic = False
n_elem = 8

vertices, elem_to_vertex, boundary_vertices, \
get_elem_ref, (vertex_map, edge_map, face_map) = \
          uniform_nodes_3d(n_elem,L[0],L[1],L[2],
                           get_elem_ref=True,
                           periodic=True)

In [ ]:
for k,v in face_map.iteritems():
    assert k[0]<k[1]
    assert k[1]<k[2]
    assert k[2]<k[3]
    assert v[0]<v[1]
    assert v[1]<v[2]
    assert v[2]<v[3]
    
for k,v in edge_map.iteritems():
    assert k[0]<k[1]
    assert v[0]<v[1]

In [ ]:
topo = SHex()
basis = LobattoBasisHex(topo, order)
#basis = LagrangeBasisHex(topo, order)
jacb = topo.calc_jacb(vertices[elem_to_vertex])
jacb_det = topo.calc_jacb_det(jacb)
jacb_inv = topo.calc_jacb_inv(jacb)
mesh = Mesh(topo, basis)
mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

In [ ]:
if periodic:
    mesh.apply_dof_maps(vertex_map, edge_map, face_map)
    mesh.boundary_dofs = np.array([0], dtype=np.int)
mesh.reorder_dofs()

In [ ]:
def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,3))
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]
    u = x*(x-x_max)*y*(y-y_max)*z*(z-z_max)
    #u += x+y+z
    #u += 1.0
    return u.reshape(shape)

def f2(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,3))
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]
    u = 2*y*(y-y_max)*x*(x-x_max)+\
        2*z*(z-z_max)*x*(x-x_max)+\
        2*y*(y-y_max)*z*(z-z_max)
    return -u.reshape(shape)

k1 = 1.0
k2 = 1.0
k3 = 1.0
def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,3))
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]
    sol  = np.sin(k1*2*np.pi*x/x_max)
    sol *= np.sin(k2*2*np.pi*y/y_max)
    sol *= np.sin(k3*2*np.pi*z/z_max)
    return sol.reshape(shape)

def f2(X):
    sol  = -f(X)
    sol *=  (k1*2*np.pi/x_max)**2\
           +(k2*2*np.pi/y_max)**2\
           +(k3*2*np.pi/z_max)**2
    return -sol

u = np.zeros(mesh.n_dofs)
Kloc = poisson_Kloc(basis, jacb_det[0], jacb_inv[0])
K, b = simple_assembly(mesh, Kloc, u)
b[mesh.boundary_dofs] = 0.0
rhs = simple_build_rhs(topo, basis, mesh, f2)
rhs += b
rhs[mesh.boundary_dofs] = 0.0
if periodic:
    rhs -= np.mean(rhs)

In [ ]:
if False:
    plt.spy(K)
np.sum(np.abs((K-K.T).data)>1e-15)

In [ ]:
ml = ruge_stuben_solver(K)
residuals = []
sol = ml.solve(rhs, tol=1e-12, residuals=residuals, maxiter=5000,
              accel='cg')
sol[mesh.boundary_dofs] = u[mesh.boundary_dofs]
if periodic:
    sol -= np.mean(sol)
len(residuals), residuals[-1]

In [ ]:
if basis.is_nodal:
    dof_phys = mesh.get_dof_phys()
    Mloc = poisson_Kloc(basis, jacb_det[0], jacb_inv[0])
    M = simple_assembly(mesh, Mloc, has_boundary=False)
    err = f(dof_phys)-sol
    print np.max(np.abs(err))
    print err.dot(M.dot(err))

In [ ]:
n = 100
x_vals = np.linspace(0,x_max,n)
y_vals = np.ones_like(x_vals)*y_max/4
z_vals = np.ones_like(x_vals)*z_max/4

phys = np.zeros((len(x_vals),3), dtype=np.double)
phys[:,0] = x_vals
phys[:,1] = y_vals
phys[:,2] = z_vals

elem, ref = get_elem_ref(phys)
Z1 = mesh.eval_elem_ref(sol, elem, ref)
Z2 = f(phys)

plt.plot(x_vals, Z1)
plt.plot(x_vals, Z2)

np.max(np.abs(Z1-Z2))